### Priprava podatkov

In [45]:
import pandas as pd

def v_uro(series):
    ure = []
    for i, ele in series.items():
        ure.append(int(ele.split(":")[0]))
    return pd.Series(ure, dtype="int")

def v_mesec(series):
    mesec = []
    for i, ele in series.items():   
        mesec.append(int(str("{:.3f}".format(ele))[:-5])) 
    return pd.Series(mesec, dtype="int")

def v_leto(series):
    leto = []
    for i, ele in series.items():
        leto.append(int(str("{:.3f}".format(ele).replace(".",""))[-4:]))
    return pd.Series(leto, dtype="int")

df = None
for year in range(2012, 2022):
    df_temp = pd.read_csv("data/data" + str(year) + ".csv", delimiter=";", encoding="utf8", low_memory=False)
    df_temp.iloc[:, 0] = df_temp.iloc[:, 0] + year * 1000_000
    df_temp.insert(1, "LetoStoritve", v_leto(df_temp.iloc[:, 1]), True)
    df_temp.iloc[:, 2] = v_mesec(df_temp.iloc[:, 2])
    df_temp.iloc[:, 3] = v_uro(df_temp.iloc[:, 3])
    df_temp.iloc[:, 4] = df_temp.iloc[:, 4].replace(["PONEDELJEK", "TOREK", "SREDA", "ČETRTEK", "PETEK", "SOBOTA", "NEDELJA"], [1, 2, 3, 4, 5, 6, 7]).astype(int)
    df_temp.iloc[:, 6] = df_temp.iloc[:, 6].replace(["DA", "NE"], [1, 0]).astype("bool")
    df_temp.iloc[:, 9] = df_temp.iloc[:, 9].replace(["GOSPODARSKA", "SPLOŠNA"], [1, 0]).astype("bool")
    df_temp.iloc[:, 10] = df_temp.iloc[:, 10].replace(["ORGANIZIRANA", " "], [1, 0]).astype("bool")
    df_temp.iloc[:, 11] = df_temp.iloc[:, 11].replace(["MLADOLETNIŠKA", " "], [1, 0]).astype("bool")
    df_temp.iloc[:, 12] = df_temp.iloc[:, 12].replace(["DA", "NE"], [1, 0]).astype("bool")
    df_temp.iloc[:, 32] = df_temp.iloc[:, 32].replace(["DA", "NE"], [1, 0]).astype("bool")

    df = pd.concat([df, df_temp], ignore_index = True)

df.iloc[:, 0:-1]

,ZaporednaStevilkaKD,LetoStoritve,MesecStoritve,UraStoritve,DanVTednu,PUStoritveKD,Povratnik,OpisKD,PoglavjeKD,GospodarskiKriminal,...,VrstaZakljucnegaDokumenta,ZaporednaStevilkaOsebeVKD,VrstaOsebe,StarostniRazred,Spol,Drzavljanstvo,Poskodba,VplivAlkohola,VplivMamil,OrganiziranaZdruzba
0,2012000001,2000,1,23,6,DIREKTORAT ZA UPRAVNE NOTRANJE ZADEVE,False,KZ08/100-/2// - GENOCID,KD ZOPER ČLOVEČNOST,False,...,OVADBA,1001.0,ŽRTEV,44-54,MOŠKI,SLOVENSKO,BREZ POŠKODBE,NN,NN,False
1,2012000002,2004,8,23,1,PU KRANJ,False,KZ08/204-/2/S/ - TATVINA,KD ZOPER PREMOŽENJE,False,...,OVADBA,2001.0,NEOVADENI OSUMLJENEC (ARHIV),18-24,MOŠKI,SLOVENSKO,NaN,NE,NE,False
2,2012000002,2004,8,23,1,PU KRANJ,False,KZ08/204-/2/S/ - TATVINA,KD ZOPER PREMOŽENJE,False,...,OVADBA,2002.0,NEOVADENI OSUMLJENEC (ARHIV),18-24,MOŠKI,SLOVENSKO,NaN,NE,NE,False
3,2012000003,2005,4,16,1,PU LJUBLJANA,False,KZ08/205-/1/1 S/ - VELIKA TATVINA,KD ZOPER PREMOŽENJE,False,...,POROČILO,3001.0,ŽRTEV,34-44,MOŠKI,SLOVENSKO,BREZ POŠKODBE,NN,NN,False
4,2012000004,2006,2,14,4,PU LJUBLJANA,False,KZ08/217-/2// - PRIKRIVANJE,KD ZOPER PREMOŽENJE,False,...,OVADBA,4001.0,OVADENI OSUMLJENEC,24-34,MOŠKI,SLOVENSKO,NaN,NE,NE,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243421,2021050281,2021,7,19,5,PU LJUBLJANA,False,KZ12/296*/1// - NASILNIŠTVO,KD ZOPER JAVNI RED IN MIR,False,...,OVADBA,50281001.0,ŽRTEV,NAD 64,MOŠKI,SLOVENSKO,BREZ POŠKODBE,NN,NN,False
1243422,2021050281,2021,7,19,5,PU LJUBLJANA,False,KZ12/296*/1// - NASILNIŠTVO,KD ZOPER JAVNI RED IN MIR,False,...,OVADBA,50281002.0,OVADENI OSUMLJENEC,54-64,MOŠKI,SLOVENSKO,NaN,NN,NN,False
1243423,2021050282,2021,10,12,4,PU LJUBLJANA,False,KZ12/192*/2// - ZANEMARJANJE MLADOLETNE OSEBE ...,"KD ZOPER ZAKONSKO ZVEZO, DRUŽINO IN OTROKE",False,...,OVADBA,50282001.0,OVADENI OSUMLJENEC,44-54,MOŠKI,TUJE,NaN,NE,NE,False
1243424,2021050282,2021,10,12,4,PU LJUBLJANA,False,KZ12/192*/2// - ZANEMARJANJE MLADOLETNE OSEBE ...,"KD ZOPER ZAKONSKO ZVEZO, DRUŽINO IN OTROKE",False,...,OVADBA,50282002.0,ŽRTEV,00-07,MOŠKI,SLOVENSKO,BREZ POŠKODBE,NN,NN,False


### 1. Ali sej je karantena spremenila pogostost družinskega nasilja?

In [46]:
df_prva = pd.DataFrame(columns=df.columns.values.tolist())
df_druga = pd.DataFrame(columns=df.columns.values.tolist())
df_spl  = pd.DataFrame(columns=df.columns.values.tolist())


In [63]:
#Graf prve in druge karantene, ter splošnega povprečja
#4.2020 - 5.2020, 11.2020 - 1.2021#

df_prva.drop(df_prva.index,inplace=True)
df_druga.drop(df_druga.index,inplace=True)
df_spl.drop(df_spl.index,inplace=True)
    
df_prva = df.loc[(4 <= df['MesecStoritve']) & (df['MesecStoritve'] <= 5) & (df['LetoStoritve'] == 2020)]
df_druga = df.loc[(11 <= df['MesecStoritve']) & (df['LetoStoritve'] == 2020)]
df_spl = df.loc[~(((4 <= df['MesecStoritve']) & (df['MesecStoritve'] <= 5) & (df['LetoStoritve'] == 2020)) & ((11 <= df['MesecStoritve']) & (df['LetoStoritve'] == 2020)))]

print(len(df_prva))
print(len(df_druga))
print(len(df_spl))

df_prva.iloc[:, 0:-1]




C:\Users\Patrik\AppData\Local\Temp\ipykernel_12964\2690787830.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prva.drop(df_prva.index,inplace=True)
C:\Users\Patrik\AppData\Local\Temp\ipykernel_12964\2690787830.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_druga.drop(df_druga.index,inplace=True)


12917
11515
1243426


,ZaporednaStevilkaKD,LetoStoritve,MesecStoritve,UraStoritve,DanVTednu,PUStoritveKD,Povratnik,OpisKD,PoglavjeKD,GospodarskiKriminal,...,VrstaZakljucnegaDokumenta,ZaporednaStevilkaOsebeVKD,VrstaOsebe,StarostniRazred,Spol,Drzavljanstvo,Poskodba,VplivAlkohola,VplivMamil,OrganiziranaZdruzba
1069009,2020006914,2020,5,0,1,PU LJUBLJANA,False,KZ12/191*/1// - NASILJE V DRUŽINI,"KD ZOPER ZAKONSKO ZVEZO, DRUŽINO IN OTROKE",False,...,OVADBA,6914001.0,OVADENI OSUMLJENEC,24-34,MOŠKI,SLOVENSKO,NaN,NE,NE,False
1069010,2020006914,2020,5,0,1,PU LJUBLJANA,False,KZ12/191*/1// - NASILJE V DRUŽINI,"KD ZOPER ZAKONSKO ZVEZO, DRUŽINO IN OTROKE",False,...,OVADBA,6914002.0,ŽRTEV,24-34,ŽENSKI,SLOVENSKO,BREZ POŠKODBE,NN,NN,False
1100553,2020025756,2020,5,17,3,PU KRANJ,False,KZ12/186*/1// - NEUPRAVIČENA PROIZVODNJA IN PR...,KD ZOPER ČLOVEKOVO ZDRAVJE,False,...,OVADBA,25756001.0,OVADENI OSUMLJENEC,34-44,MOŠKI,SLOVENSKO,NaN,NE,NE,False
1103855,2020027491,2020,4,0,2,PU KRANJ,False,KZ12/191*/1// - NASILJE V DRUŽINI,"KD ZOPER ZAKONSKO ZVEZO, DRUŽINO IN OTROKE",False,...,OVADBA,27491001.0,OVADENI OSUMLJENEC,24-34,MOŠKI,SLOVENSKO,NaN,NE,NE,False
1103856,2020027491,2020,4,0,2,PU KRANJ,False,KZ12/191*/1// - NASILJE V DRUŽINI,"KD ZOPER ZAKONSKO ZVEZO, DRUŽINO IN OTROKE",False,...,OVADBA,27491002.0,ŽRTEV,24-34,ŽENSKI,SLOVENSKO,BREZ POŠKODBE,NN,NN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243240,2021050172,2020,4,23,5,PU LJUBLJANA,False,KZ12/283*/1// - KRIVA OVADBA,KD ZOPER PRAVOSODJE,False,...,OVADBA,50172002.0,ŽRTEV,54-64,MOŠKI,SLOVENSKO,BREZ POŠKODBE,NN,NN,False
1243241,2021050172,2020,4,23,5,PU LJUBLJANA,False,KZ12/283*/1// - KRIVA OVADBA,KD ZOPER PRAVOSODJE,False,...,OVADBA,50172003.0,ŽRTEV,54-64,ŽENSKI,SLOVENSKO,BREZ POŠKODBE,NN,NN,False
1243418,2021050280,2020,4,8,5,PU LJUBLJANA,False,KZ12/122*/4// - LAHKA TELESNA POŠKODBA,KD ZOPER ŽIVLJENJE IN TELO,False,...,POROČILO,50280001.0,ŽRTEV,NAD 64,MOŠKI,TUJE,LAHKA TELESNA POŠKODBA,NN,NN,False
1243419,2021050280,2020,4,8,5,PU LJUBLJANA,False,KZ12/122*/4// - LAHKA TELESNA POŠKODBA,KD ZOPER ŽIVLJENJE IN TELO,False,...,POROČILO,50280002.0,OSTALO,54-64,MOŠKI,SLOVENSKO,NaN,NN,NN,False


### 2. Ali se je med karanteno spremenila pogostost drugih kaznivih dejanj?

### 3. Kako je uvedba policijske ure vplivala na kriminal v nočnem času?